In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

In [1]:
CLIENT_ID = 'FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3' # your Foursquare ID
CLIENT_SECRET = 'CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

credentails:
CLIENT_ID: FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3
CLIENT_SECRET:CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B


In [2]:
neighbourhood_latitude = 25.1921465
neighbourhood_longitude = 66.5949955

neighbourhood_name = 'Karachi, Sindh' # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Karachi, Sindh are 25.1921465, 66.5949955.


In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FJBBHL0CQFIZKY0C3JXRMJ1Q3YYQRG0W5OGAREGYE1TMYRL3&client_secret=CM0T2DDI4IXNZAAVZNHW2Z1JQ1QYQQXUYF55GTMH4OF3BY0B&v=20180605&ll=25.1921465,66.5949955&radius=500&limit=100'

In [5]:
results = requests.get(url).json()

NameError: name 'requests' is not defined